# Local PageRank - Local similarities in the nodes of a network

### Description

[PageRank](https://en.wikipedia.org/wiki/PageRank) is an algorithm, originally proposed by Google’s founders Brin and Page, that assigns a numerical weighting (centrality) to each node in a network with the purpose of measuring its relative importance in the network. To do so, PageRank exploits the direct and indirect paths in the network, as well as the importance (weights) of links, in a Markov chain setting. 

PageRank can be personalised to detect centrality (similarity) with respect to a specific topic (i.e., a set of nodes), as opposed to the entire network. 

We talk of Local PageRank when the personalised PageRank is measuring centrality (similarity) with respect to a single node. Local PageRank is a local measure that can be efficiently calculated locally with controlled precision [1], i.e., without the need of surfing the entire network. 

Local PageRank was also used as an effective measure for clustering applications [2,3,4].



[1] Andersen, Chung, Lang. 2006. Local graph partitioning using pagerank vectors. http://www.leonidzhukov.net/hse/2015/networks/papers/andersen06localgraph.pdf

[2] Avrachenkov, Dobrynin, Nemirovsky, Pham, Smirnova. 2008. Pagerank based clustering of hypertext document collections. https://dl.acm.org/citation.cfm?id=1390549

[3] Cho and MuLee. 2010. Authority-shift clustering: Hierarchical clustering by authority seeking on graphs. https://ieeexplore.ieee.org/document/5540081

[4] Tabrizi, Shakery, Asadpour, Abbasi, Tavallaie. 2013. Personalized pagerank clustering: A graph clustering algorithm based on random walks. https://www.sciencedirect.com/science/article/pii/S0378437113006316?via%3Dihub



### Assignments

1. Implement a scalable procedure that for a given network, identified by its (weighted) adjacency matrix A, extracts the Local PageRank information among nodes up to a certain precision (e.g., 1%). In doing so, exploit the results of [2]. To ensure scalability, consider extracting the Local PageRank information only for those nodes which are neighbours (i.e., for which a link is active in matrix A is active), in such a way that the resulting Local PageRank information can be collected in a matrix L which is active where the adjacency matrix A is active. Incidentally observe that matrix L is sparse if A is sparse, and that L is not a symmetric matrix.

2. Possibly propose ideas to speed up the algorithm (e.g., neural network approaches applied to the local network portions)

3. Apply the algorithm to small and large networks, with the aim of evaluating its scalability properties. Use networks available in the web, e.g., https://snap.stanford.edu/data .

4. Propose a (simple) clustering algorithm that exploits the Local PageRank matrix L, and test it on a few networks. You can take inspiration from [3,4,5] for this. Datasets for testing can be found at http://cs.joensuu.fi/sipu/datasets .
